## Setting up g2o

g2o is an open-source C++ framework for optimizing graph-based nonlinear error functions. We will be using g2o's binaries in this project.

More regarding g2o from their homepage:
>A wide range of problems in robotics as well as in computer-vision involve the minimization of a non-linear error function that can be represented as a graph. Typical instances are simultaneous localization and mapping (SLAM) or bundle adjustment (BA). The overall goal in these problems is to find the configuration of parameters or state variables that maximally explain a set of measurements affected by Gaussian noise. g2o is an open-source C++ framework for such nonlinear least squares problems. 

Note that the instructions are for Linux and if you are using any other OS, you'll have to go through the official pages.

Please go through [the official page first](https://github.com/RainerKuemmerle/g2o). If you are stuck, only then refer to the following points. Please install the requirements carefully before actual compilation.

### Requirements
As written [there](https://github.com/RainerKuemmerle/g2o#requirements), cmake & libeigen3-dev.

Both of them can be installed via apt-get. Some useful commands:

```
sudo apt-get -y install cmake
which cmake #check the installation
cmake --version
sudo apt-get install libeigen3-dev
```

`libeigen3-dev` will reside in `usr/include/eigen3/` if you followed the above. "Optional requirements" are not necessary for this project, but the 4 packages mentioned there can be installed via `apt-get` if you are interested. For example,
```
sudo apt-get install libsuitesparse-dev
```
 
### Compilation
First clone the repository, then `cd` into it and follow the 4 steps [there](https://github.com/RainerKuemmerle/g2o#compilation). 

```
mkdir build
cd build
cmake ../
make
```

After the above is successfully done, we want to call it from command-line via `g2o`. To do that, add the following to your `~/.bashrc`:
```
export PATH=$PATH:/your/path/here/g2o/bin/
```
For example,
```
export PATH=$PATH:/home/shubodh/packages/g2o/bin/
```

You can then call `g2o` from within your code via (You can instead even use `g2o_viewer` which allows you to do more than the following command, described in next section):
```
def g2o_call():
    cmd = "g2o -o opt.g2o given.g2o"
    os.system(cmd)
```
`given.g2o` is your given input to g2o optimizer and `opt.g2o` is its output. How to generate `given.g2o`?
The given `edges.txt` is already in g2o format. Just rename it to `edges.g2o`, but it in its current state has only 1 pose (first). You have to generate the rest of the poses using the motion model (see main notebook) and write the rest of the poses just below the first pose. A sample g2o file is as follows:
```
VERTEX_SE2 0 -5.0 -8.0 0.0
VERTEX_SE2 1 -4.55 -8.0 0.0
...
EDGE_SE2 0 1 0.4500000000000002 0.0 0.0 500.0 0.0 0.0 500.0 0.0 500.0
EDGE_SE2 1 2 0.4500000000000002 0.0 0.0 500.0 0.0 0.0 500.0 0.0 500.0
...
FIX 0
EDGE_SE2 0 80 -0.9499999999999993 -0.1999999999999993 0.0 700.0 0.0 0.0 700.0 0.0 700.0
EDGE_SE2 2 82 -0.75 -0.1999999999999993 0.0 700.0 0.0 0.0 700.0 0.0 700.0

```

G2O's format is explained [link here](https://www.notion.so/saishubodh/G2O-Edge-Description-fa07cc28967541dc8a71170de46c5da7).

### g2o viewer

Using this viewer, you can see pose graph along with the loop closure edges. You can also directly optimize instead of using the command described above.

```g2o_viewer file.g2o``` 

Controls: arrow keys/mouse

It has various options which you can play around with before clicking on "Optimize". Don't worry, you are not expected to understand every option available there. But for questions asked, you will have to play around a bit before getting good results. You can see if it is "good results" or not from the image that I inserted in the main notebook.

# Questions/Problems?

If you face any problems during installation, try out `sudo apt update` and `sudo apt upgrade` and reinstall. 

After succesful compilation, if you are unable to run g2o_viewer, first check if your path has been modified. This can be done using `echo $PATH`.

### g2o read/write helper functions

In [ ]:
def readVertex(fileName):
    f = open(fileName, 'r')
    A = f.readlines()
    f.close()

    x_arr = []
    y_arr = []
    theta_arr = []

    for line in A:
        if "VERTEX_SE2" in line:
            (ver, ind, x, y, theta) = line.split()
            x_arr.append(float(x))
            y_arr.append(float(y))
            theta_arr.append(float(theta.rstrip('\n')))

    return jnp.array([x_arr, y_arr, theta_arr])

def readEdge(fileName):
    f = open(fileName, 'r')
    A = f.readlines()
    f.close()

    ind1_arr = []
    ind2_arr = []
    del_x = []
    del_y = []
    del_theta = []

    for line in A:
        if "EDGE_SE2" in line:
            (edge, ind1, ind2, dx, dy, dtheta, _, _, _, _, _, _) = line.split()
            ind1_arr.append(int(ind1))
            ind2_arr.append(int(ind2))
            del_x.append(float(dx))
            del_y.append(float(dy))
            del_theta.append(float(dtheta))

    return (jnp.array( ind1_arr), jnp.array(ind2_arr), jnp.array(del_x), jnp.array(del_y), jnp.array(del_theta))